<a href="https://colab.research.google.com/github/supervik/binance-trading-performance/blob/master/binance_trading_performance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Binance trading performance evaluation

## User settings

In [1]:
# @markdown *API keys. Use read-only API keys for more security:*
API_key = 'm5gNycphXSKHEUT3yCOs875VqXzEGSudt1RGpML9O3PkxVCVNUODIUMiBe8PaTTi ' #@param {type:"string"}
API_secret = 'PRgpBJLfzWgnuyZ9OjfA2ODJk8v0YLyhyANXgtzYeud4b6dBGjMzM7EQDK4b5g0o' #@param {type:"string"}

# @markdown *Market. Use dash '-' between base and quote assets.*
market = 'MFT-ETH' #@param {type:"string"}
 
# @markdown *Start and end date (UTC). Note, that **end_date** is used for getting final prices. P&L greatly depends on this date!*
start_date = "2020-06-01" #@param {type:"date"}
end_date = '2020-06-30' #@param {type:"date"}

## Binance API

In [2]:
#@title
# Credits to @Bablofil https://github.com/Bablofil/binance-api
import time
import json
import urllib
import hmac, hashlib
import requests

from urllib.parse import urlparse, urlencode
from urllib.request import Request, urlopen

class Binance():
    
    methods = {
            #  Public methods     
            'ping':             {'url': 'ping', 'method': 'GET', 'private': False},                     
            'time':             {'url': 'time', 'method': 'GET', 'private': False},         
            'exchangeInfo':     {'url': 'exchangeInfo', 'method': 'GET', 'private': False},
            'depth':            {'url': 'depth', 'method': 'GET', 'private': False},
            'trades':           {'url': 'trades', 'method': 'GET', 'private': False},  
            'historicalTrades': {'url': 'historicalTrades', 'method': 'GET', 'private': False},  
            'aggTrades':        {'url': 'aggTrades', 'method': 'GET', 'private': False},
            'klines':           {'url': 'klines', 'method': 'GET', 'private': False},   
            'avgPrice':         {'url': 'avgPrice', 'method': 'GET', 'private': False}, 
            'ticker24hr':       {'url': 'ticker/24hr', 'method': 'GET', 'private': False},
            'tickerPrice':      {'url': 'ticker/price', 'method': 'GET', 'private': False},   
            'tickerBookTicker': {'url': 'ticker/bookTicker', 'method': 'GET', 'private': False}, 
            #  Private methods   
            'createOrder':      {'url': 'order', 'method': 'POST', 'private': True},                        
            'testOrder':        {'url': 'test', 'method': 'POST', 'private': True},              
            'orderInfo':        {'url': 'order', 'method': 'GET', 'private': True},
            'cancelOrder':      {'url': 'order', 'method': 'DELETE', 'private': True},
            'openOrders':       {'url': 'openOrders', 'method': 'GET', 'private': True}, 
            'allOrders':        {'url': 'allOrders', 'method': 'GET', 'private': True},     
            'account':          {'url': 'account', 'method': 'GET', 'private': True}, 
            'myTrades':         {'url': 'myTrades', 'method': 'GET', 'private': True}, 
   }
    
    def __init__(self, API_KEY, API_SECRET):
        self.API_KEY = API_KEY
        self.API_SECRET = bytearray(API_SECRET, encoding='utf-8')
        self.shift_seconds = 0

    def __getattr__(self, name):
        def wrapper(*args, **kwargs):
            kwargs.update(command=name)
            return self.call_api(**kwargs)
        return wrapper

    def set_shift_seconds(self, seconds):
        self.shift_seconds = seconds
        
    def call_api(self, **kwargs):

        command = kwargs.pop('command')
        api_url = 'https://api.binance.com/api/v3/' + self.methods[command]['url']

        payload = kwargs
        headers = {}
        
        payload_str = urllib.parse.urlencode(payload)
        if self.methods[command]['private']:
            payload.update({'timestamp': int(time.time() + self.shift_seconds - 1) * 1000})
            payload_str = urllib.parse.urlencode(payload).encode('utf-8')
            sign = hmac.new(
                key=self.API_SECRET,
                msg=payload_str,
                digestmod=hashlib.sha256
            ).hexdigest()

            payload_str = payload_str.decode("utf-8") + "&signature="+str(sign) 
            headers = {"X-MBX-APIKEY": self.API_KEY}

        if self.methods[command]['method'] == 'GET':
            api_url += '?' + payload_str

        # print(api_url, payload_str, self.methods[command])
        response = requests.request(
            method=self.methods[command]['method'], 
            url=api_url, 
            data="" if self.methods[command]['method'] == 'GET' else payload_str,
            headers=headers)

        if 'code' in response.text:
            print(response.text)
            response = response.json()
            return response['error']
        return response.json()



## Calculations

In [3]:
#@title
import sys
import time
import calendar
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_rows', None)

try:
    asset_base = market.split("-")[0]
    asset_quote = market.split("-")[1]
except IndexError:
    raise Exception(f"!!! Warning: Use dash '-' to split base and quote assets for {market}!")

symbol = asset_base + asset_quote

# Connect to Binance
binance = Binance(API_key, API_secret)
try:
    trades = binance.myTrades(symbol = symbol)
    print(f"INFO: Exchange is successfully connected.")
except:
    raise Exception(f"!!! Warning: Can't get orders for {symbol}! Read message above.")    

# Create DataFrame 
df = pd.DataFrame(trades, columns = ['time', 'symbol', 'isBuyer', 'price', 'qty', 'quoteQty', 'commission', 'commissionAsset'])
qty_base = 'qty_' + asset_base
qty_quote = 'qty_' + asset_quote
df.columns = ['time', 'symbol', 'side', 'price', qty_base, qty_quote, 'fee', 'fee_coin']
df.side = df.side.replace([True, False], [1, -1])
df = df.astype({'price': 'float', qty_base: 'float', qty_quote: 'float', 'fee': 'float'})

# Start from time
time_format = '%Y-%m-%d'
start_date_ms = int(calendar.timegm(time.strptime(start_date, time_format)) * 1000)
end_date_ms = int((calendar.timegm(time.strptime(end_date, time_format))) * 1000)
df = df[(df.time >= start_date_ms) & (df.time <= end_date_ms + 86_400_000)]
df.time = pd.to_datetime(df.time, unit='ms')

# Find time for getting market prices
time_now = time.gmtime(time.time())
day_now_ms = calendar.timegm((time_now.tm_year, time_now.tm_mon, time_now.tm_mday, 0, 0, 0, 0, 0, 0)) * 1000
prices_time = min(day_now_ms, end_date_ms)

# Get symbol price
try:
    symbol_price  = float(binance.klines(symbol = symbol, interval = '1m', startTime = prices_time, limit = 1)[0][1])
except:
    print(f"Something wrong with request of {symbol} price. Please try again.")

# Get quote-USD price
if asset_quote == 'USDC' or asset_quote == 'USDT' or asset_quote == 'BUSD':
    usd_price = 1
else:
    try:
        usd_price  = float(binance.klines(symbol = asset_quote + 'USDT', interval = '1m', startTime = prices_time, limit = 1)[0][1])
    except:
        print(f"Something wrong with the request of {asset_quote}USDT price. Please try again.")

# Get BNB-quote price
if asset_quote == 'BNB':
    bnb_price = 1
else:
    try:
        bnb_price = float(binance.klines(symbol = 'BNB' + asset_quote, interval = '1m', startTime = prices_time, limit = 1)[0][1])
    except:
        print(f"Something wrong with the request of BNB{asset_quote} price. Please try again.")

# Summary
days = int((prices_time - start_date_ms)/(1000 * 86400)) + 1
average_buy = df[df.side == 1][qty_quote].sum()/df[df.side == 1][qty_base].sum()
average_sell = df[df.side == -1][qty_quote].sum()/df[df.side == -1][qty_base].sum()
total_volume = df[qty_quote].sum()

# Delta
delta_base = (df[qty_base] * df.side).sum()
delta_quote = - (df[qty_quote] * df.side).sum()

# Fees
fee_bnb = df[df.fee_coin == 'BNB'].fee.sum()
fee_base = df[df.fee_coin == asset_base].fee.sum()
fee_quote = df[df.fee_coin == asset_quote].fee.sum()

# Totals
total_quote = (delta_base - fee_base) * symbol_price + (delta_quote - fee_quote) - fee_bnb * bnb_price
total_usd = total_quote * usd_price
prices_time_utc = time.strftime(time_format, time.gmtime(prices_time/1000))

INFO: Exchange is successfully connected.


## Trades

In [4]:
#@title
if df.empty:
  print(f"No trades found for {symbol} from {start_date} till {end_date}")
else: 
  print(f"\nTrades gathered for {symbol}:")
  df.side = df.side.replace([1, -1], ['BUY', 'SELL'])
  df = df.reset_index(drop=True)
  df


Trades gathered for MFTETH:


,time,symbol,side,price,qty_MFT,qty_ETH,fee,fee_coin
0,2020-06-16 13:18:03.591,MFTETH,BUY,0.000004,182557.0,0.750309,0.008125,BNB
1,2020-06-16 13:32:08.095,MFTETH,SELL,0.000004,145347.0,0.606097,0.006486,BNB
2,2020-06-16 13:32:08.171,MFTETH,SELL,0.000004,37210.0,0.155166,0.001661,BNB
3,2020-06-30 16:41:21.043,MFTETH,BUY,0.000004,44320.0,0.163098,0.001812,BNB
4,2020-06-30 16:41:21.828,MFTETH,BUY,0.000004,499158.0,1.836901,0.020411,BNB
5,2020-06-30 16:56:26.149,MFTETH,SELL,0.000004,460893.0,1.709913,0.018710,BNB
6,2020-06-30 16:56:26.157,MFTETH,SELL,0.000004,82585.0,0.306390,0.003353,BNB
7,2020-06-30 19:12:17.470,MFTETH,BUY,0.000004,560224.0,2.000000,0.022290,BNB
8,2020-06-30 19:20:31.977,MFTETH,SELL,0.000004,560224.0,2.000000,0.021950,BNB
9,2020-06-30 21:09:30.598,MFTETH,BUY,0.000004,550964.0,1.999999,0.022380,BNB


## Summary

In [5]:
#@title
print(f"Summary for {symbol} for period [{start_date} - {end_date}]:")
print(f"   Days: {days}")
print(f"   Trades executed: {df.time.count()}")
print(f"   Total volume traded ({asset_quote}): {round(total_volume, 8)}")
print(f"   Average buy price: {round(average_buy, 8)}")
print(f"   Average sell price: {round(average_sell, 8)}")
print(f"\nTrading delta:")
print(f"   Delta {asset_base}: {round(delta_base, 8)}")
print(f"   Delta {asset_quote}: {round(delta_quote, 8)}")
print(f"\nFees:")
print(f"   Fees {asset_base}: {round(fee_base, 8)}")
print(f"   Fees {asset_quote}: {round(fee_quote, 8)}")
print(f"   Fees BNB: {round(fee_bnb, 8)}")
print(f"\nPrices at the end of the period [{prices_time_utc}]:")
print(f"   Price {symbol}: {symbol_price}")
print(f"   Price {asset_quote}USDT: {usd_price}")
print(f"   Price BNB{asset_quote}: {bnb_price}")
print(f"\nTotal profit:")
print(f"   Total profit ({asset_quote}): {round(total_quote, 8)}")
print(f"   Total profit (USDT): {round(total_usd, 8)}")

Summary for MFTETH for period [2020-06-01 - 2020-06-30]:
   Days: 30
   Trades executed: 12
   Total volume traded (ETH): 13.53338202
   Average buy price: 3.67e-06
   Average sell price: 3.69e-06

Trading delta:
   Delta MFT: 0.0
   Delta ETH: 0.0327674

Fees:
   Fees MFT: 0.0
   Fees ETH: 0.0
   Fees BNB: 0.14918898

Prices at the end of the period [2020-06-30]:
   Price MFTETH: 3.89e-06
   Price ETHUSDT: 227.98
   Price BNBETH: 0.067842

Total profit:
   Total profit (ETH): 0.02264612
   Total profit (USDT): 5.16286272


## Formulas:


$P_{Q} = (\Delta B - fee_B)\cdot price_{BQ} + (\Delta Q - fee_Q) - fee_{bnb}\cdot price_{bnb}$

$P_{USD} = P_{Q}\cdot price_{usd} $

*   $P_{Q}$ - total profit in quote currency
*   $P_{USD}$ - total profit in USDT currency
*   $\Delta B = b - b_0,$ - base currency delta
*   $\Delta Q = q - q_0$ - quote currency delta
*   $b_0, b, q_0, q$ - amount of base/quote currency at the start and at the end of the period 
*   $fee_B, fee_Q, fee_{bnb}$ - fees in base, quote and BNB currencies
*   $price_{BQ}$ - market price of base-quote symbol at the end of the period
*   $price_{bnb}$ - market price of BNB-quote symbol at the end of the period
*   $price_{usd}$ - market price of quote-USDT symbol at the end of the period

This calculation is based upon hummingbot performance measuring:<br>
https://hummingbot.io/blog/2019-07-measure-performance-crypto-trading/